In [1]:
#Step 1: Load the relevant libraries and packages

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [2]:
#Reading the PDF docs

loader = PyPDFDirectoryLoader("/Users/rajinipreethajohn/Desktop/Langchain/huggingface/us_census")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000,chunk_overlap =200)
final_documents = text_splitter.split_documents(documents)
len(final_documents)

316

In [3]:
#Embedding using HuggingFaceBgeEmbeddings

huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5", #"sentence-transformers/all-MiniLM-L6-v2"
    model_kwargs={"device": "cpu"}, 
    encode_kwargs={'normalize_embeddings':True},
    
)


/Users/rajinipreethajohn/Desktop/Langchain/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import numpy as np
page_content = huggingface_embeddings.embed_query(final_documents[0].page_content)
#np.array(page_content).shape (will give 384 embeddings in final_documents[0]
np.array(page_content)


array([-8.46568495e-02, -1.19099151e-02, -3.37892473e-02,  2.94559430e-02,
        5.19160032e-02,  5.73839471e-02, -4.10018452e-02,  2.74267569e-02,
       -1.05128251e-01, -1.58055816e-02,  7.94858411e-02,  5.64318746e-02,
       -1.31765064e-02, -3.41544151e-02,  5.81605313e-03,  4.72547859e-02,
       -1.30746579e-02,  3.12997657e-03, -3.44226025e-02,  3.08406167e-02,
       -4.09086309e-02,  3.52738425e-02, -2.43761409e-02, -4.35831100e-02,
        2.41503529e-02,  1.31986691e-02, -4.84447787e-03,  1.92347262e-02,
       -5.43913208e-02, -1.42735019e-01,  5.15522342e-03,  2.93115899e-02,
       -5.60810231e-02, -8.53531621e-03,  3.14140730e-02,  2.76736263e-02,
       -2.06187833e-02,  8.24231580e-02,  4.15424742e-02,  5.79654984e-02,
       -3.71587090e-02,  6.26162253e-03, -2.41389461e-02, -5.61793940e-03,
       -2.51715127e-02,  5.04967384e-03, -2.52801031e-02, -2.91950209e-03,
       -8.24041106e-03, -5.69604822e-02,  2.30822917e-02, -5.54216560e-03,
        5.11555411e-02,  

In [5]:
#Vector Store creation
# If errror with FAISS, THEN 'pip install faiss-cpu'


VectorStore = FAISS.from_documents(final_documents[:250],huggingface_embeddings)


In [6]:
# Query using Similarity Search

query = "WHAT IS HEALTH INSURANCE COVERAGE??"
relevant_documents=VectorStore.similarity_search(query)
print(relevant_documents[0].page_content)


2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [7]:
#In order to get more than one relevant answer, we could use 
#retriever and specify how many we want

retriever=VectorStore.as_retriever(search_type="similarity",search_kwargs={"k":3}) #relevant 3 answers)
                                   
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2c5922c10> search_kwargs={'k': 3}


In [17]:
#In order to get the relevant retrievers we need to connect to the
#LLM we will use hf token

import os

from dotenv import load_dotenv
load_dotenv()
api_key= os.environ.get("HF_API_TOKEN")


In [38]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs={"temperature":0.1,"max_length":500},huggingfacehub_api_token=api_key,

)
query="What is health insurance coverage?"
hf.invoke(query)

'What is health insurance coverage?\n\nHealth insurance coverage is a contract between an insurance company and an individual or group that outlines the terms of the insurance policy. The policy specifies the types of medical services and treatments that are covered, as well as the financial responsibilities of both the insured person and the insurance company.\n\nHealth insurance coverage can help protect individuals and families from the financial burden of medical expenses, which can be significant. By paying a premium, or monthly fee, the insured person can access healthcare'

In [40]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [41]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [42]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [43]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [44]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to < www.
census.gov/topics/health/health-insurance/
guidance.html >.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distr